Copyright (c) 2024 Habana Labs, Ltd. an Intel Company.

#### Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Using Parameter Efficient Fine Tuning on Llama 2 with 7B Parameters on One Intel&reg; Gaudi&reg; 2 AI Accelerator
This example will Fine Tune the Llama2-7B model using Parameter Efficient Fine Tuning (PEFT) and then run inference on a text prompt.  This will be using the Llama2 model with two task examples from the Optimum Habana library on the Hugging Face model repository.   The Optimum Habana library is optimized for Deep Learning training and inference on First-gen Gaudi and Gaudi2 and offers tasks such as text generation, language modeling, question answering and more. For all the examples and models, please refer to the [Optimum Habana GitHub](https://github.com/huggingface/optimum-habana#validated-models).

This example will Fine Tune the Llama2-7B model using Parameter Efficient Fine Tuning (PEFT) on the timdettmers/openassistant-guanaco dataset using the Language-Modeling Task in Optimum Habana.

In [ ]:
# Start with the `exit()` command to re-start the Python kernel to ensure that there are no other proceses holding the Intel Gaudi Accelerator as you start to run this notebook. You will see a warning that the kernel has died, this is expected.
exit()

### Parameter Efficient Fine Tuning with Low Rank Adaptation
Parameter Efficient Fine Tuning is a strategy for adapting large pre-trained language models to specific tasks while minimizing computational and memory demands.   It aims to reduce the computational cost and memory requirements associated with fine-tuning large models while maintaining or even improving their performance.  It does so by adding a smaller task-specific layer, leveraging knowledge distillation, and often relying on few-shot learning, resulting in efficient yet effective models for various natural language understanding tasks.   PEFT starts with a pre-trained language model that has already learned a wide range of language understanding tasks from a large corpus of text data. These models are usually large and computationally expensive.   Instead of fine-tuning the entire pre-trained model, PEFT adds a task-specific layer or a few task-specific layers on top of the pre-trained model. These additional layers are relatively smaller and have fewer parameters compared to the base model.


In [ ]:
%cd ~/Gaudi-tutorials/PyTorch/Single_card_tutorials

### Model Setup: 

##### Install the Parameter Efficient Fine Tuning Library methods
This is taking the PEFT method from the Hugging Face repository and will be used to help create the PEFT Fine Tuning with the Llama2 model.

In [ ]:
import sys
!{sys.executable} -m pip install peft==0.10.0

##### Install the Optimum-Habana Library

In [ ]:
!{sys.executable} -m pip install -q optimum-habana==1.14.1

##### Pull the Hugging Face Examples from GitHub
These contain the working Hugging Face Task Examples that have been optimized for Gaudi.  For Fine Tuning, we'll use the language-modeling task. 

In [ ]:
%cd ~
!git clone -b v1.14.1 https://github.com/huggingface/optimum-habana.git

##### Go to the Language Modeling Task and install the model specific requirements

In [ ]:
%cd ~/optimum-habana/examples/language-modeling
!pip install --quiet -r requirements.txt

##### How to access and Use the Llama 2 model - Get a Security Token 

Use of the pretrained model is subject to compliance with third party licenses, including the “Llama 2 Community License Agreement” (LLAMAV2). For guidance on the intended use of the LLAMA2 model, what will be considered misuse and out-of-scope uses, who are the intended users and additional terms please review and read the instructions in this link https://ai.meta.com/llama/license/.
Users bear sole liability and responsibility to follow and comply with any third party licenses, and Habana Labs disclaims and will bear no liability with respect to users’ use or compliance with third party licenses.

To be able to run gated models like the Meta Llama model you need to get a security token:
- Have a HuggingFace account  
- Read more about Hugging Face [security tokens](https://huggingface.co/docs/hub/en/security-tokens)
- Agree to the terms of use of the Llama 2 model in its [model card](https://huggingface.co/meta-llama/Llama-2-7b-hf) on the HF Hub
- set a read token and copy this in place of the `<your_token_here>` in the cell below
- Login to your account using the HF CLI: run huggingface-cli login before launching your script

In [ ]:
!huggingface-cli login --token <your_token_here>

## Fine Tuning the model with PEFT and LoRA

We'll now run the fine tuning with the PEFT method. Remember that the PEFT methods only fine-tune a small number of extra model parameters, thereby greatly decreasing the computational and storage costs. Recent State-of-the-Art PEFT techniques achieve performance comparable to that of full fine-tuning.

##### Here's a summary of the Fine Tuning command in the cell below. 
1. Using the language modeling with LoRA; `run_lora_clm.py`; It's very efficient: only 0.06% of the total parameters are being fine tuned of the total 7B parameters.
2. Only 3 epochs are needed for fine tuning, it takes less than 20 minutes to run with the openassisant-guanaco dataset.
3. There are some Intel Gaudi specific run commands here, including `use_habana` to ensure the workload runs on Intel Gaudi, `use_lazy_mode` to ensure a more efficient lazy execution.  You can refer to the details in the [Hugging Face examples](https://github.com/huggingface/optimum-habana/tree/main/examples/language-modeling) for more background on each of the model parameters. 


In [ ]:
!python3 run_lora_clm.py \
    --overwrite_output_dir=True \
    --model_name_or_path meta-llama/Llama-2-7b-hf \
    --dataset_name timdettmers/openassistant-guanaco \
    --bf16 True \
    --output_dir ~/optimum-habana/examples/language-modeling/model_lora_llama_single \
    --num_train_epochs 3 \
    --per_device_train_batch_size 16 \
    --evaluation_strategy "no" \
    --save_strategy "no" \
    --learning_rate 1e-4 \
    --warmup_ratio  0.03 \
    --lr_scheduler_type "constant" \
    --max_grad_norm  0.3 \
    --logging_steps 1 \
    --do_train \
    --do_eval \
    --use_habana \
    --use_lazy_mode \
    --throughput_warmup_steps 3 \
    --lora_rank=8 \
    --lora_alpha=16 \
    --lora_dropout=0.05 \
    --lora_target_modules "q_proj" "v_proj" \
    --dataset_concatenation \
    --report_to none \
    --max_seq_length 512 \
    --low_cpu_mem_usage True \
    --validation_split_percentage 4 \
    --adam_epsilon 1e-08

#### LoRA Fine Tuning Completed
You will now see a "model_lora_llama_single" folder created which contains the PEFT model `adapter_model.safetensors` which will be used in the inference example below. 

In [ ]:
%ls -lS ~/optimum-habana/examples/language-modeling/model_lora_llama_single

## Inference with Llama 2

We'll now use the Hugging Face `text-generation` task to run inference on the Llama2-7b model; we'll generate text based on an included prompt.  Notice that we've included a path to the PEFT model that we've just created.

First, we'll move to the text-generation examples folder and install the requirements. 

In [ ]:
%cd ~/optimum-habana/examples/text-generation
!pip install --quiet -r requirements.txt

You will see that we are now running inference with the `run_generation.py` task and we are including the PEFT model that we Fine Tuned in the steps above. 

```
--prompt "I am a dog. Please help me plan a surprise birthday party for my human, including fun activities, games and decorations. And don't forget to order a big bone-shaped cake for me to share with my fur friends!" \
--peft_model /root/Gaudi-tutorials/PyTorch/Single_card_tutorials/optimum-habana/examples/language-modeling/model_lora_llama_single
```

In [ ]:
# Use the prompt above to compare between the PEFT and non-PEFT examples
prompt = input("Enter a prompt for text generation: ")

Before running the model, there are some Intel Gaudi accelerator specific items:
* The options `use_hpu_graphs` and `use_kv_cache` are set for Intel Gaudi performance. All Llama based Hugging Face models support HPU-graphs and KV-cache optimizations  
* The `max_new_tokens` can be set to control the length of the output generated by the run_generation.py script

In [ ]:
!python3 run_generation.py \
    --model_name_or_path meta-llama/Llama-2-7b-hf \
    --batch_size 1 \
    --do_sample \
    --max_new_tokens 300 \
    --use_hpu_graphs \
    --use_kv_cache \
    --bf16 \
    --prompt "{prompt}" \
    --peft_model ~/model_lora_llama_single

##### Comparison without PEFT and LoRA
In this example, we're simply running the Llama2 7B model **without** including the PEFT fine tuned model, so that you are losing the additional detail that is brought to the model, and the results have signficantly less information and fidelity compared to the last model.

In [ ]:
!python3 run_generation.py \
    --model_name_or_path meta-llama/Llama-2-7b-hf \
    --batch_size 1 \
    --do_sample \
    --max_new_tokens 300 \
    --use_hpu_graphs \
    --use_kv_cache \
    --bf16 \
    --prompt "{prompt}"

In [ ]:
# Please be sure to run this exit command to ensure that the resources running on Intel Gaudi are released 
exit()